In [ ]:
#These are the imports needed to use the following functions
import os
import pandas as pd
#this will connect to Codeup mysql server if this function is within the env.py file in directory
from env import host, username, password


def get_connection(db, user=username, host=host, password=password):
    
    '''
    This function is to connect to the Codeup MySQL server, and by itself won't do anything. It works in conjunction with 
    the  other functions within this .py file.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_titanic_data():

    '''
   This function will check locally if there's a titanic.csv file in the local directory, and if not, working with the 
    get_connection function, will pull the titanic dataset from the Codeup MySQL server. After that, it will also save a copy of 
    the csv locally if there wasn't one, so it doesn't have to run the query each time.
    '''
    if os.path.isfile('titanic.csv'):
        return pd.read_csv('titanic.csv')
    else:
        url = get_connection('titanic_db')
        query = '''
                SELECT *
                FROM passengers
                '''
        titanic = pd.read_sql(query, url)
        titanic.to_csv('titanic.csv')
        return titanic

def get_iris_data():
    '''
    This function will check locally if there's a iris.csv file in the local directory, and if not, working with the 
    get_connection function, will pull the iris dataset from the Codeup MySQL server. After that, it will also save a copy of 
    the csv locally if there wasn't one, so it doesn't have to run the query each time.
    '''
    if os.path.isfile('iris.csv'):
        return pd.read_csv('iris.csv')
    else:
        url = get_connection('iris_db')
        query = '''
                SELECT *
                FROM species
                JOIN measurements USING(species_id);
                '''
        iris = pd.read_sql(query, url)
        iris.to_csv('iris.csv')
        return iris

def get_telco_data():
    '''
    This function will check locally if there's a telco.csv file in the local directory, and if not, working with the 
    get_connection function, will pull the telco dataset from the Codeup MySQL server. After that, it will also save a copy of 
    the csv locally if there wasn't one, so it doesn't have to run the query each time.
    '''
    if os.path.isfile('telco.csv'):
        return pd.read_csv('telco.csv')
    else:
        url = get_connection('telco_churn')
        query = '''
                SELECT *
                FROM customers
                JOIN contract_types USING(contract_type_id)
                JOIN internet_service_types USING (internet_service_type_id)
                JOIN payment_types types USING(payment_type_id);
                '''
        telco = pd.read_sql(query, url)
        telco.to_csv('telco.csv')
        return telco
    
#this will return the data from specifically the attendance table in the tidy data dataset
def get_attendance_data():
    '''
    This function will check locally if there's a attendance.csv file in the local directory, and if not, working with the 
    get_connection function, will pull the attendance dataset from the tidy_data database in the Codeup MySQL server. After that,
    it will also save a copy of the csv locally if there wasn't one, so it doesn't have to run the query each time.
    '''
    if os.path.isfile('attendance.csv'):
        return pd.read_csv('attendance.csv')
    else:
        url = get_connection('tidy_data')
        query = '''
                SELECT *
                FROM attendance
                '''
        attendance = pd.read_sql(query, url)
        attendance.to_csv('attendance.csv')
        attendance.drop(columns='Unnamed: 0.1')
        return attendance
    


In [ ]:
#These are the imports needed to run the functions within this file
import os
import pandas as pd
from sklearn.model_selection import train_test_split
#this will connect to Codeup mysql server if this function is within the env.py file in directory
from env import host, username, password

def split_train_test(df, col, seed=42):
    '''
    This function will split a dataset into train, validate, and test variables to model with. Make sure to assign to three 
    variables when running.
    '''
    seed = 123
    train, val_test = train_test_split(df, train_size=.6, random_state=123, stratify=df[col])
    validate, test = train_test_split(val_test, train_size=.6, random_state=123, stratify=val_test[col])
    
    return train, validate, test

def prep_iris(df):
    '''
    This function is used to prepare the iris dataset. It will drop several unneeded columns, rename the 'species_name' to just 
    'species', and create and concatenate dummies.
    '''
    to_drop = ['species_id', 'measurement_id', 'Unnamed: 0']
    df.drop(columns=to_drop, inplace=True)
    df = df.rename(columns={'species_name':'species'})
    
    dummies = pd.get_dummies(df[['species']], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    
    return df

def prep_titanic(df):
    '''
    This is used to prepare the titanic dataset to work with. It will drop unneeded columns, as well as create and then 
    concatenate dummies onto the DataFrame
    '''
    to_drop = ['Unnamed: 0', 'class', 'embarked', 'passenger_id', 'deck', 'age']
    df.drop(columns=to_drop, inplace=True)
    
    dummies = pd.get_dummies(df[['sex', 'embark_town']], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    
    df = df.drop(columns=['sex', 'embark_town'])
    
    return df

def prep_telco(df):
    '''
    This function will prepare the telco_churn dataset for further use. It will convert the total_charges column into the float 
    data type, create various dummies and concatenate those dummies, and then drop unneeded columns.
    '''
    telco['total_charges'] = telco['total_charges'].replace(' ', '0')
    telco['total_charges'] = telco['total_charges'].astype(float)
    
    to_dummy = ['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 
                'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 
                'paperless_billing', 'contract_type', 'internet_service_type', 
                'payment_type']
    dummies = pd.get_dummies(df[to_dummy], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    
    drop = ['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 
                'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 
                'paperless_billing', 'contract_type', 'internet_service_type', 
                'payment_type', 'Unnamed: 0', 'payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id']
    df.drop(columns=drop, inplace=True)
                 
    return df